In [150]:
import numpy as np
import pandas as pd
import copy

In [151]:
def normal_equation(X,Y):
  W = np.transpose(X)
  W = np.dot(W,X)
  # print("determinant: ",np.linalg.det(W));
  W = np.linalg.inv(W)
  W = np.dot(W,np.transpose(X))
  W = np.dot(W,Y)
  return W
  

In [152]:
def normal_equation_with_regularization(X,Y,lamb):
  W = np.transpose(X)
  W = np.dot(W,X)
  one = np.identity(X.shape[1])
  one[0, 0] = 0
  W = W + lamb*one
  W = np.linalg.inv(W)
  W = np.dot(W,np.transpose(X))
  W = np.dot(W,Y)
  return W

In [153]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IML datasets/Housing Price data set.csv")
df = df.drop(['Unnamed: 0'], axis=1)
m = df.shape[0]
n = 4
X = np.array([np.ones(m, np.float32), df['lotsize'].to_numpy(),df['bedrooms'].to_numpy(), df['bathrms'].to_numpy()]).T
Y = df['price'].to_numpy()
X_train, X_test, Y_train, Y_test = X[:382], X[382:], Y[:382], Y[382:]

a).  Normal equations  with  and without regularization and compare their performances in terms of % error in prediction.

In [154]:
print("Normal Equation without Regularization:")
W = normal_equation(X_train,Y_train)
print()
print("Weights without regularization: ", W)
temp = np.dot(X_test, W)
error = abs(Y_test - temp)/Y_test
error = np.sum(error)/164
print()
print("Error without regularization: ",error*100,"%")
print()

Normal Equation without Regularization:

Weights without regularization:  [-4.89492370e+03  5.93101241e+00  5.52857201e+03  1.90100176e+04]

Error without regularization:  18.666490014537153 %



In [169]:
lamb = [0.01,0.1,1,5,10,25,50,90,100]
print("Normal Equation with Regularization:\n")
cnt=1
for l in lamb:
  print(cnt,". ")
  W = normal_equation_with_regularization(X_train,Y_train,l)
  print("Weights with regularization: ", W)
  temp = np.dot(X_test, W)
  error = abs(Y_test - temp)/Y_test
  error = np.sum(error)/164
  print()
  print("Error with regularization(lambda = : ",l,"): ",error*100,"%")
  print()
  cnt+=1


Normal Equation with Regularization:

1 . 
Weights with regularization:  [-4.89345776e+03  5.93113372e+00  5.52878565e+03  1.90079252e+04]

Error with regularization(lambda = :  0.01 ):  18.666398680295625 %

2 . 
Weights with regularization:  [-4.88027124e+03  5.93222442e+00  5.53070263e+03  1.89891168e+04]

Error with regularization(lambda = :  0.1 ):  18.665577642838638 %

3 . 
Weights with regularization:  [-4.74908590e+03  5.94302649e+00  5.54931487e+03  1.88032408e+04]

Error with regularization(lambda = :  1 ):  18.657462199267325 %

4 . 
Weights with regularization:  [-4.18050351e+03  5.98885079e+00  5.62062896e+03  1.80228754e+04]

Error with regularization(lambda = :  5 ):  18.625887429784058 %

5 . 
Weights with regularization:  [-3.50092027e+03  6.04161991e+00  5.68703068e+03  1.71410143e+04]

Error with regularization(lambda = :  10 ):  18.600616738322802 %

6 . 
Weights with regularization:  [-1.64237792e+03  6.17613332e+00  5.77638338e+03  1.49783222e+04]

Error with reg

b). Design Predictor using Batch Gradient Descent Algorithm, Stochastic Gradient Algorithm and mini batch Gradient Descent algorithms (determining minibatch size is your choice- here it could be 10, 20, 30 etc.) with and without feature scaling and compare their performances in terms of % error in prediction.

In [156]:
def batch_gradient_descent(X,Y,alpha,itr):
  W = np.zeros((n, 1))
  for i in range(0,itr):
     H = np.dot(W.T, X)
     Z = H - Y
     dW = np.dot(X, Z.T)
     W = W - (alpha*dW)/(X.shape[1])
     J = np.sum(Z**2)/(2*m)
  return W

In [157]:
def scaling(X):
  X1 = copy.deepcopy(X)
  stdv = np.std(X1[:, 0:], axis=0)
  stdv[0] = 1
  mean = np.mean(X1[:, 0:], axis=0)
  mean[0] = 0
  X1[:, 0:] = (X1[:, 0:]-mean) / stdv
  return X1,mean,stdv

In [158]:
def batch_gradient_descent_with_scaling(X_train,Y_train,X_test,Y_test,alpha,itr):
  X_train, MEAN, STDV = scaling(X_train)
  X_test = (X_test - MEAN) / STDV
  X_train, X_test = X_train.T, X_test.T
  W = np.zeros((n, 1))
  for i in range(0, itr):
      H = np.dot(W.T, X_train)
      Z = H - Y_train
      dW = np.dot(X_train, Z.T)
      W = W - (alpha*dW)/(X_train.shape[1])
      J = np.sum(Z**2)/(2*m)
  print("Weights with scaling: \n", W)
  print()
  pred = np.dot(W.T, X_test)
  error = abs(Y_test - pred)/Y_test
  error = np.sum(error, axis=1)/X_test.shape[1]
  print("Error with Scaling: ",error*100,"%")

In [159]:
print("Batch Gradient Descent Without Scaling:")
print()
W = batch_gradient_descent(X_train.T,Y_train,0.000000001, 10000)
print("Weights without scaling:\n", W)
print()
pred = np.dot(W.T, X_test.T)
error = abs(Y_test - pred)/Y_test
error = np.sum(error, axis=1)/X_test.T.shape[1]
print("Error without Scaling: ",error*100,"%")

Batch Gradient Descent Without Scaling:

Weights without scaling:
 [[ 0.04491122]
 [12.51410683]
 [ 0.17650582]
 [ 0.09924596]]

Error without Scaling:  [29.91677804] %


In [160]:
print("Batch Gradient Descent With Scaling:")
print()
batch_gradient_descent_with_scaling(X_train,Y_train,X_test,Y_test,0.01, 10000)

Batch Gradient Descent With Scaling:

Weights with scaling: 
 [[63936.35340314]
 [12011.48915726]
 [ 4370.56419059]
 [ 9851.34899367]]

Error with Scaling:  [18.66649001] %


In [161]:
def stochastic_gradient_descent(X,Y,alpha,itr):
  W = np.zeros((n, 1))
  for k in range(0,itr):
      random = np.random.permutation(X.shape[1])
      X_shuffle,Y_shuffle = X[:,random],Y[random]
      for i in range(0,X.shape[1]):
        x = X_shuffle[:, i].reshape((-1, 1))
        y = Y_shuffle[i]
        H = np.dot(W.T, x)
        Z = H - y
        dW = x*Z
        W = W - alpha*dW
  return W

In [162]:
def stochastic_gradient_descent_with_scaling(X_train,Y_train,X_test,Y_test,alpha,itr):
  X_train, MEAN, STDV = scaling(X_train)
  X_test = (X_test - MEAN) / STDV
  X_train, X_test = X_train.T, X_test.T
  W = np.zeros((n, 1))
  for k in range(0, itr):
      random = np.random.permutation(X_train.shape[1])
      X_shuffle,Y_shuffle = X_train[:,random],Y_train[random]
      for i in range(0,X_train.shape[1]):
        x = X_shuffle[:, i].reshape((-1, 1))
        y = Y_shuffle[i]
        H = np.dot(W.T, x)
        Z = H - y
        dW = x*Z
        W = W - alpha*dW
  print("Weights with scaling: \n", W)
  print()
  pred = np.dot(W.T, X_test)
  error = abs(Y_test - pred)/Y_test
  error = np.sum(error, axis=1)/X_test.shape[1]
  print("Error with Scaling: ",error*100,"%")

In [163]:
print("Stochastic Gradient Descent Without Scaling:")
print()
W = stochastic_gradient_descent(X_train.T,Y_train,0.000000001, 10000)
print("Weights without scaling: \n", W)
print()
pred = np.dot(W.T, X_test.T)
error = abs(Y_test - pred)/Y_test
error = np.sum(error, axis=1)/X_test.T.shape[1]
print("Error without Scaling: ",error*100,"%")

Stochastic Gradient Descent Without Scaling:

Weights without scaling: 
 [[16.37049285]
 [12.1799133 ]
 [65.03401314]
 [36.89751075]]

Error without Scaling:  [29.77521275] %


In [164]:
print("Stochastic Gradient Descent With Scaling:")
print()
stochastic_gradient_descent_with_scaling(X_train,Y_train,X_test,Y_test,0.01, 10000)

Stochastic Gradient Descent With Scaling:

Weights with scaling: 
 [[63808.06593379]
 [11961.09324613]
 [ 2649.77427355]
 [ 9326.15731255]]

Error with Scaling:  [18.52232213] %


In [165]:
def create(X, Y, sizes):
    mini_batches = []
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    num_batches = X.shape[0] // sizes
    i=0
    for i in range(0,num_batches):
      excerpt = indices[i*sizes:(i+1)*sizes]
      X_mini,Y_mini = X[excerpt],Y[excerpt]
      mini_batches.append((X_mini.T,Y_mini))

    if X.shape[0] % sizes != 0:
      excerpt = indices[i*sizes:(i+1)*sizes]
      X_mini, Y_mini = X[excerpt], Y[excerpt]
      mini_batches.append((X_mini.T, Y_mini))
    return mini_batches

In [166]:
def mini_batch_gradient_descent(X,Y,alpha,itr,sizes):
  W = np.zeros((n, 1))
  for i in range(0, itr):
      mini_batches = create(X.T,Y,sizes)
      for mini_batch in mini_batches:
        X_mini,Y_mini = mini_batch
        H = np.dot(W.T, X_mini)
        Z = H - Y_mini
        dW = np.dot(X_mini,Z.T)
        W = W - (alpha*dW)/X_mini.shape[1]
  return W

In [167]:
def mini_batch_gradient_descent_with_scaling(X_train,Y_train,X_test,Y_test,alpha,itr,sizes):
  X_train, MEAN, STDV = scaling(X_train)
  X_test = (X_test - MEAN) / STDV
  X_train, X_test = X_train.T, X_test.T
  W = np.zeros((n, 1))
  for i in range(0, itr):
      mini_batches = create(X_train.T,Y_train,sizes)
      for mini_batch in mini_batches:
        X_mini,Y_mini = mini_batch
        H = np.dot(W.T, X_mini)
        Z = H - Y_mini
        dW = np.dot(X_mini,Z.T)
        W = W - (alpha*dW)/X_mini.shape[1]
  print("Weights with Scaling: \n", W)
  print()
  pred = np.dot(W.T, X_test)
  error = abs(Y_test - pred)/Y_test
  error = np.sum(error, axis=1)/X_test.shape[1]
  print("Error with Scaling,(Batch Size = ",sizes,"): ",error*100,"%")

In [170]:
batch = [10,20,30,40,50]
cnt=1
print("Mini Batch Gradient Descent With Scaling:")
print()
for b in batch:
  print(cnt,". ")
  W = mini_batch_gradient_descent(X_train.T,Y_train, 0.00000001, 10000, b)
  print("Weights without Scaling: \n", W)
  print()
  pred = np.dot(W.T, X_test.T)
  error = abs(Y_test - pred)/Y_test
  error = np.sum(error, axis=1)/X_test.T.shape[1]
  print("Error without Scaling: ",error*100,"%")
  print()
  mini_batch_gradient_descent_with_scaling(X_train,Y_train,X_test,Y_test, 0.01, 10000, b)
  cnt+=1
  print()

Mini Batch Gradient Descent With Scaling:

1 . 
Weights without Scaling: 
 [[16.68940469]
 [13.04351817]
 [66.29299968]
 [37.61821996]]

Error without Scaling:  [30.3117148] %

Weights with Scaling: 
 [[63927.94372129]
 [11925.73298022]
 [ 4101.37318798]
 [ 9721.87583572]]

Error with Scaling,(Batch Size =  10 ):  [18.59869597] %

2 . 
Weights without Scaling: 
 [[ 8.56344829]
 [13.47317054]
 [34.03523884]
 [19.302457  ]]

Error without Scaling:  [31.05436088] %

Weights with Scaling: 
 [[63965.4099079 ]
 [11951.43234455]
 [ 4333.97386923]
 [ 9775.1804128 ]]

Error with Scaling,(Batch Size =  20 ):  [18.61464485] %

3 . 
Weights without Scaling: 
 [[ 5.56182862]
 [12.67644664]
 [22.10659631]
 [12.54130562]]

Error without Scaling:  [29.97878906] %

Weights with Scaling: 
 [[63896.58841718]
 [11785.5280599 ]
 [ 4330.86236784]
 [10156.28572184]]

Error with Scaling,(Batch Size =  30 ):  [18.65006266] %

4 . 
Weights without Scaling: 
 [[ 4.27903984]
 [11.98409262]
 [17.01314142]
 [ 9.648